In [9]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [10]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jrumm1s5
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jrumm1s5


In [11]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [12]:
%%cu
#include <cstdlib>
#include <iostream>
#include <time.h>

using namespace std;

__global__ void kernel(bool* adj_mat, const int N, bool* visited, int* frontier, bool* new_frontier){
	int row_idx = frontier[blockIdx.x+1]; 
	long offset = N * row_idx;

	// update new_frontier in threads
	int col_idx = threadIdx.x;	
	if(adj_mat[offset + col_idx] && !visited[col_idx]){
		new_frontier[col_idx] = true;
	}
}
__global__ void k2(const int N, bool* visited, int* frontier, bool* new_frontier){
	int cn = 0;
	for(int i=0;i<N;i++){
		if(new_frontier[i]){
			new_frontier[i] = false;
			frontier[++cn] = i;
			visited[i] = true;
		}
	}
	frontier[0] = cn;
}

int main(int arg, char** argv){
 

	const int N = 15;
	const int EDGE_NUM = 25; // exist_edges / N^2
	cout<<"generating graph of size "<<N<<"; edge number: "<<EDGE_NUM<<endl;

	int** adj_mat = (int**)malloc(N*sizeof(int*));
	for(int row=0; row<N; row++)
		adj_mat[row] = (int*)malloc(N*sizeof(int));
	for(int i=0;i<N;i++)
		for(int j=0;j<N;j++)
			adj_mat[i][j]  = 0;

	int n = EDGE_NUM;
	while(n>0){
		int u = rand()%N;
		int v = rand()%N;
		if(adj_mat[u][v]==0 && u!=v){
			adj_mat[u][v] = 1;
			n--;
		}
	}

	for(int i=0;i<N;i++){
		for(int j=0;j<N;j++){
			cout<<adj_mat[i][j]<<' ';
		}
		cout<<endl;
	}



	//read graph from <input>.txt
	bool* h_adj_mat = (bool*)malloc(N*N*sizeof(bool));
	for(int i=0;i<N*N;i++){
		string a;
		cin>>a;
		if(a=="1") h_adj_mat[i] = true;
		else h_adj_mat[i] = false;		
	}

	//generate visited and frontier vector; init them with node 0;
	bool* h_visited = (bool*)malloc(N*sizeof(bool));
	for(int i=0;i<N;i++) h_visited[i] = false;
	int* h_frontier = (int*)malloc(N*sizeof(int));
	bool* h_new_frontier = (bool*)malloc(N*sizeof(bool));
	for(int i=0;i<N;i++) h_new_frontier[i] = false;

	h_visited[0] = true;
	h_frontier[0] = 1;
	h_frontier[1] = 0;
	
	//malloc mem in gpu
	clock_t start,end, s, e;
	start = clock();
	bool *d_adj_mat, *d_visited, *d_new_frontier;
	int *d_frontier;
	cudaMalloc((void**) &d_adj_mat, sizeof(bool) * N * N);
	cudaMemcpy((void*) d_adj_mat, (void*) h_adj_mat, sizeof(bool)*N*N, cudaMemcpyHostToDevice);
	
	cudaMalloc((void**) &d_visited, sizeof(bool) * N);
	cudaMemcpy((void*) d_visited, (void*) h_visited, sizeof(bool)*N, cudaMemcpyHostToDevice);
	
	cudaMalloc((void**) &d_frontier, sizeof(int) * (N+1));
	cudaMemcpy((void*) d_frontier, (void*) h_frontier, sizeof(int)*N, cudaMemcpyHostToDevice);
	
	cudaMalloc((void**) &d_new_frontier, sizeof(bool) * N);
	cudaMemcpy((void*) d_new_frontier, (void*) h_new_frontier, sizeof(bool)*N, cudaMemcpyHostToDevice);

	//loop until frontier vector is empty 
	int cn =1;
	double t=0;
	while(h_frontier[0]!=0){
		cn+=h_frontier[0];
		//lauch kernel : launch threads to update frontier_len, visited and frontier in gpu local mem
		s= clock();
		kernel<<<h_frontier[0], N>>>(d_adj_mat,N,d_visited,d_frontier, d_new_frontier);
		
		k2<<<1,1>>>(N, d_visited,d_frontier, d_new_frontier);
		e=clock();
		t+=double(e-s);

		cudaMemcpy((void*) h_frontier, (void*) d_frontier, sizeof(int)*1, cudaMemcpyDeviceToHost);
	}
	end = clock();

	cout << "parallel BFS uses " << double(end - start) << " us in total"<< endl;



	return 0;
}

generating graph of size 15; edge number: 25
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 
0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 
0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 
0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 
parallel BFS uses 252707 us in total

